In [29]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from mirisim.config_parser import SimulatorConfig
from mirisim import MiriSimulation
import tso_img_sims_setup
from tso_img_sims_setup import wasp103_scene, wasp103_sim_config

from importlib import reload

In this notebook I'm going to generate simulated MIRI time series imaging data, for data management & pipeline testing purposes. To install Mirisim, see the [the public release webpage](http://miri.ster.kuleuven.be/bin/view/Public/MIRISim_Public). The target for the mock observations is WASP-103, an exoplanet host star with the following properties from [the exoplanet encyclopaedia](http://exoplanet.eu/catalog/wasp-103_b/):

* spectral type F8V
* T_bb = 6110 K
* V = 12.0, K = 10.7

K magnitude of 10.7 corresponds to a flux of 32.5 mJy or 32.5e3 microJy.

Using the ETC, I calculated the following number of groups for a high-SNR but unsaturated image:
* FULL array: NGROUPS = 5
* SUB64 subarray: NGROUPS = 60

We want to simulate a short (NINT = 1), medium (NINT = 10) and long exposure (NINT = 500). In total that's 6 simulations, so let's order them as follows:

| Sim no | Array   | NGroups  | NInt   | NExp   |
| -------|---------| ---------|--------|--------|
|1       |FULL     |  5       | 1      | 1      |
|2       |SUB64    |  60      | 1      | 1      |
|3       |FULL     |  5       | 10     | 1      |
|4       |SUB64    |  60      | 10     | 1      |
|5       |FULL     |  5       | 500    | 1      |
|6       |SUB64    |  60      | 500    | 1      |

### Steps in setting up the simulation

This notebook will go through the following steps:

* Create the scene
* Set up the simulation
* Run the simulation

Each step has its own function. Steps 1 and 2 will each write out a .ini file, which will be used as input for the final step.

In [87]:
arr = ['FULL', 'SUB64'] * 3
ngrp = [5, 60] * 3
nints = [1, 1, 10, 10, 500, 500]


## Step 1: Creating the input scene (WASP-103)

Here we'll create the input scene for the simulations using the function wasp103_scene(). Arguments:

* scene_file: the filename for the .ini file
* write_cube: write the scene image out to a FITS file (optional; default=False)

The function returns a mirisim.skysim.scenes.CompositeSkyScene object.


In [88]:
scene_ini = wasp103_scene(scene_file='wasp103_scene.ini', write_cube=False)

2019-02-06 14:08:10,122 - INFO - Initializing Point
2019-02-06 14:08:10,126 - INFO - Initializing Background


Creating scene. Scene will be written to file wasp103_scene.ini
File wasp103_scene.ini already exists. Will be overwritten.
wasp103_scene.ini exists, overwrite (y/[n])?y


In [89]:
print(scene_ini)

wasp103_scene.ini


## Step 2: Configuring the simulation

Now I'll set up the simulations and prepare to run them. I'll set it up to loop through the 6 simulations. For this I wrote the function wasp103_sim_config. Check the docstring for descriptions and default values of the arguments. 

The function will write out another .ini file containing the simulation configuration, and it returns the output filename for further use.

In [90]:
#reload(tso_img_sims_setup)
#from tso_img_sims_setup import wasp103_sim_config

for (a, g, i) in zip(arr, ngrp, nints):
    sim_ini = wasp103_sim_config(mode='imaging', arr=a, ngrp=g, nint=i, nexp=1, filt='F770W', 
                   scene_file=scene_ini, out=False)
    print(sim_ini)

Found scene file wasp103_scene.ini
wasp103_FULL_5G1I1E_simconfig.ini
Found scene file wasp103_scene.ini
wasp103_SUB64_60G1I1E_simconfig.ini
Found scene file wasp103_scene.ini
wasp103_FULL_5G10I1E_simconfig.ini
Found scene file wasp103_scene.ini
wasp103_SUB64_60G10I1E_simconfig.ini
Found scene file wasp103_scene.ini
wasp103_FULL_5G500I1E_simconfig.ini
Found scene file wasp103_scene.ini
wasp103_SUB64_60G500I1E_simconfig.ini


### Step 3: Run the simulation

In the following step we'll run the simulations for the 6 different cases. For each run, we need 3 input files: the scene, the simulation configuration, and the simulator setup file. The first and last of these remain the same for each run, and we loop through the list of 6 simulation config files.

After the simulation has run, the code renames the output directory to include the simulation settings to the directory.


In [102]:
cfg_files = glob.glob('*_simconfig.ini')

# configure the simulator engine - this requires no editing from the default
simulator_config = SimulatorConfig.from_default()

for f in cfg_files[:2]:
    tmp = f.split('.')
    fcomps = tmp[0].split('_')
    sim = MiriSimulation.from_configfiles(f)
    sim.run()
    outdir = sorted(glob.glob('*_*_mirisim'), key=os.path.getmtime )[-1]
    new_outdir = 'wasp103_imtso_{0}_{1}_{2}'.format(fcomps[1], fcomps[2], outdir)
    os.rename(outdir, new_outdir)
    print(outdir, new_outdir)


2019-02-06 14:54:58,531 - INFO - Using simulation configuration: wasp103_FULL_5G10I1E_simconfig.ini
2019-02-06 14:54:58,536 - INFO - Using scene configuration: wasp103_scene.ini
2019-02-06 14:54:58,543 - INFO - MIRISim version: 2.0.0
2019-02-06 14:54:58,545 - INFO - MIRI Simulation started.
2019-02-06 14:54:58,549 - INFO - Output will be saved to: 20190206_145458_mirisim
2019-02-06 14:54:58,552 - INFO - Storing configs in output directory.
2019-02-06 14:54:58,559 - INFO - Using $CDP_DIR for location of CDP files: /Users/kendrew//CDP
2019-02-06 14:54:58,561 - INFO - Setting up simulated Observation, with following settings:
2019-02-06 14:54:58,563 - INFO - Configuration Path: IMA_FULL
2019-02-06 14:54:58,566 - INFO - Primary optical path: IMA
2019-02-06 14:54:58,568 - INFO - IMA Filter: F770W
2019-02-06 14:54:58,571 - INFO - IMA Subarray: FULL
2019-02-06 14:54:58,574 - INFO - IMA detector readout mode: FAST
2019-02-06 14:54:58,578 - INFO - IMA detector # exposures: 1
2019-02-06 14:54:58

2019-02-06 14:56:05,977 - INFO - Simulating 5 groups for integration 8.
2019-02-06 14:56:07,652 - INFO - Simulating 5 groups for integration 9.
2019-02-06 14:56:09,249 - INFO - Simulating 5 groups for integration 10.
2019-02-06 14:56:11,247 - INFO - Adding the DARK calibration from /Users/kendrew//CDP/MIRI_FM_MIRIMAGE_FAST_DARK_06.01.00.fits
2019-02-06 14:56:11,958 - INFO - Correcting nonlinearity from MIRI_FM_MIRIMAGE_LINEARITY_06.02.00.fits
2019-02-06 14:56:19,857 - INFO - Exposure time 138.75s (duration 139.81s) 
2019-02-06 14:56:23,606 - INFO - Wrote detector image: 20190206_145458_mirisim/det_images/det_image_seq1_MIRIMAGE_F770Wexp1.fits
2019-02-06 14:56:23,609 - INFO - MIRI Simulation finished. Results have been saved to: 20190206_145458_mirisim

2019-02-06 14:56:23,611 - INFO - Using simulation configuration: wasp103_FULL_5G1I1E_simconfig.ini
2019-02-06 14:56:23,616 - INFO - Using scene configuration: wasp103_scene.ini
2019-02-06 14:56:23,621 - INFO - MIRISim version: 2.0.0
2019

20190206_145458_mirisim wasp103_imtso_FULL_5G10I1E_20190206_145458_mirisim


2019-02-06 14:56:23,963 - INFO - Reading 'DISTORTION' model from '/Users/kendrew//CDP/MIRI_FM_MIRIMAGE_DISTORTION_7B.03.00.fits'
2019-02-06 14:56:24,257 - INFO - Reading 'DISTORTION' model from '/Users/kendrew//CDP/MIRI_FM_MIRIMAGE_DISTORTION_7B.03.00.fits'
2019-02-06 14:56:24,724 - INFO - Creating pointing for position 1
2019-02-06 14:56:24,726 - INFO - Creating exposure event for position 1
2019-02-06 14:56:24,729 - INFO - Reading 'DISTORTION' model from '/Users/kendrew//CDP/MIRI_FM_MIRIMAGE_DISTORTION_7B.03.00.fits'
2019-02-06 14:56:25,036 - INFO - Observation simulation started.
2019-02-06 14:56:25,037 - INFO - Simulating ExposureEvent for pointing 1
2019-02-06 14:56:25,039 - INFO - Simulating Imager exposures for pointing 1
2019-02-06 14:56:25,041 - INFO - Simulating detector illumination for Imager exposures for pointing 1
2019-02-06 14:56:25,042 - INFO - Running ImSim.
2019-02-06 14:56:25,043 - INFO - MirimImager processing  version=36, cfgpath=IMA_FULL
2019-02-06 14:56:25,048 -

20190206_145623_mirisim wasp103_imtso_FULL_5G1I1E_20190206_145623_mirisim
